<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# SQL Server 2019 big data cluster Tutorial
## 03 - Creating and Querying a Data Mart

In this tutorial you will learn how to create and query a Data Mart using Virtualized Data in a SQL Server big data cluster. 

Wide World Importers is interested in ingesting the data from web logs from an HDFS source where they have been streamed. They want to be able to analyze the traffic to see if there is a pattern in time, products or locations. 

The web logs, however, are refreshed periodically. WWI would like to keep the logs in local storage to do deeper analysis. 

In this Jupyter Notebook you'll create a location to store the log files as a SQL Server Table in the SQL Data Pool, and then fill it by creating an External Table that reads HDFS.

In [1]:
USE WideWorldImporters;
GO

IF NOT EXISTS(SELECT * FROM sys.external_data_sources WHERE name = 'SqlDataPool')
  CREATE EXTERNAL DATA SOURCE SqlDataPool
  WITH (LOCATION = 'sqldatapool://service-mssql-controller:8080/datapools/default');

Commands completed successfully.

Total execution time: 00:00:00.0414641

Commands completed successfully.

Total execution time: 00:00:00.0663092

In [2]:
CREATE EXTERNAL TABLE [web_clickstream_clicks_data_pool]
    ("wcs_click_date_sk" BIGINT 
    , "wcs_click_time_sk" BIGINT 
    , "wcs_sales_sk" BIGINT 
    , "wcs_item_sk" BIGINT
    , "wcs_web_page_sk" BIGINT 
    , "wcs_user_sk" BIGINT)
    WITH
    (
    DATA_SOURCE = SqlDataPool,
    DISTRIBUTION = ROUND_ROBIN
    );
GO

Commands completed successfully.

Total execution time: 00:00:09.9645892

In [3]:
/* Create an External Table that can read from the Storage Pool File Location */
IF NOT EXISTS(SELECT * FROM sys.external_tables WHERE name = 'web_clickstreams_hdfs')
BEGIN
    CREATE EXTERNAL TABLE [web_clickstreams_hdfs]
    ("wcs_click_date_sk" BIGINT 
    , "wcs_click_time_sk" BIGINT 
    , "wcs_sales_sk" BIGINT 
    , "wcs_item_sk" BIGINT
    , "wcs_web_page_sk" BIGINT 
    , "wcs_user_sk" BIGINT)
    WITH
    (
        DATA_SOURCE = SqlStoragePool,
        LOCATION = '/web_logs',
        FILE_FORMAT = csv_file
    );
END

Commands completed successfully.

Total execution time: 00:00:00.4667564

In [12]:
BEGIN
   INSERT INTO web_clickstream_clicks_data_pool
   SELECT wcs_click_date_sk
    , wcs_click_time_sk 
    , wcs_sales_sk 
    , wcs_item_sk 
    , wcs_web_page_sk  
    , wcs_user_sk 
     FROM web_clickstreams_hdfs
END

(6770549 rows affected)

Total execution time: 00:00:52.7335665

In [13]:
SELECT count(*) FROM [dbo].[web_clickstream_clicks_data_pool]
SELECT TOP 10 * FROM [dbo].[web_clickstream_clicks_data_pool]

(1 row affected)

(10 rows affected)

Total execution time: 00:00:09.2888920

(No column name)
6770549


wcs_click_date_sk,wcs_click_time_sk,wcs_sales_sk,wcs_item_sk,wcs_web_page_sk,wcs_user_sk
37735,67197,NULL,8769,53,NULL
37735,66368,NULL,4710,53,NULL
37735,29134,NULL,2568,53,NULL
37736,62336,NULL,11924,53,NULL
37736,61470,NULL,3914,53,NULL
37736,6060,NULL,16280,53,NULL
37736,32023,NULL,12267,53,NULL
37736,49579,NULL,15327,53,NULL
37736,56435,NULL,15305,53,NULL
37736,7883,NULL,4853,53,NULL


## Next Steps: Continue on to Working with Spark and ETL

Now you're ready to open the next Python Notebook - `bdc_tutorial_04.ipynb` - to learn how to create and work with Spark and Extracting, Transforming and Loading data.